In [1]:
import tensorflow as tf
import os
import cv2
import glob
import random
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm  # For progress tracking

In [5]:
import tensorflow as tf
import os
import cv2
import numpy as np

data_groups = ["HC", "Left_MAS", "Right_MAS"]
data_groups = list(data_groups)

data_dir = '../../output/'  # Change the path to your dataset location
image_label_list = []

def display_image_name(image_path):
    # Extract the file name from the path
    image_name = os.path.basename(image_path)

    # Display the image name
    print(f"File name: {image_name}")

def rotate_images(img):
    X_rotate = []
    k = tf.constant(1, dtype=tf.int32)  # Initialize k with a constant value
    tf_img = tf.image.rot90(img, k=k)

    # Resize the original image to the desired shape
    img_resized = tf.image.resize(img, (640, 480))
    X_rotate.append(img_resized.numpy())

    for i in range(3):  # Rotation at 90, 180, and 270 degrees
        rotated_img = tf.image.rot90(img, k=i + 1)
        # Resize the rotated image to the desired shape
        rotated_img_resized = tf.image.resize(rotated_img, (640, 480))
        X_rotate.append(rotated_img_resized.numpy())

    X_rotate = np.array(X_rotate, dtype=np.float32)
    return X_rotate

# Use os.walk to traverse all subdirectories and process PNG files
for group in data_groups:
    # Define the path to your data group folder
    data_group_path = os.path.join(data_dir, group)

    for root, dirs, files in os.walk(data_group_path):
        for file in files:
            if file.lower().endswith(".png"):
                file_path = os.path.join(root, file)

                # Read the image using cv2
                image = cv2.imread(file_path)
                resized_image = cv2.resize(image, (640, 480))
                # JPEG compression
                encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 90]  # Adjust quality as needed
                _, compressed_image = cv2.imencode('.jpg', resized_image, encode_param)
                compressed_image = compressed_image.tobytes()

                # Reshape the compressed image to have height, width, and channels
                compressed_image = tf.image.decode_jpeg(compressed_image, channels=3)

                # Combine resizing and rotation
                augmented_images = rotate_images(compressed_image)

                label = 0
                tri_label = 0

                if group == 'HC':
                    label = 1
                    tri_label = 1
                elif group == 'Left_MAS':
                    label = 2
                    tri_label = 2
                elif group == 'Right_MAS':
                    label = 2
                    tri_label = 3

                # Append the original and augmented images to the list
                image_label_list.append((compressed_image.numpy(), str(label), str(tri_label)))
                image_label_list.extend([(augmented_image, label, tri_label) for augmented_image in augmented_images])



In [6]:
image_label_list

[]

In [21]:
# # Initialize lists for training and testing data
train_images = []
train_labels = []
train_tri_labels = []
test_images = []
test_labels = []
test_tri_labels = []

random.shuffle(image_label_list)
# Calculate the split point
split_point = int(0.7 * len(image_label_list))

# Split the data into training and testing sets
train_data = image_label_list[:split_point]
test_data = image_label_list[split_point:]

# Append the training data to the respective lists
for data in train_data:
    train_labels.append(data[1])  
    train_tri_labels.append(data[2])
    train_images.append(data[0])  # Image
    
# Append the testing data to the respective lists
for data in test_data:
    test_labels.append(data[1])  
    test_tri_labels.append(data[2])
    test_images.append(data[0])  # Image

In [22]:
label_binarizer = LabelBinarizer()


train_one_hot_labels = label_binarizer.fit_transform(train_labels)
test_one_hot_labels = label_binarizer.fit_transform(test_labels)
train_tri_one_hot_labels = label_binarizer.fit_transform(train_tri_labels)
test_tri_one_hot_labels = label_binarizer.fit_transform(test_tri_labels)



In [24]:
# Function to convert an image into a feature
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# Create TFRecord writer for training data
train_writer = tf.io.TFRecordWriter('train.tfrecord')

# Iterate through the training data and write to TFRecords
for image, label,tri_label in tqdm(zip(train_images, train_one_hot_labels, train_tri_one_hot_labels), total=len(train_images)):
    feature = {
        'image': _bytes_feature(image),
        'label': _int64_feature(label),
        'tri_label': _int64_feature(tri_label)
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    train_writer.write(example.SerializeToString())

train_writer.close()

# Create TFRecord writer for testing data
test_writer = tf.io.TFRecordWriter('test.tfrecord')

# Iterate through the testing data and write to TFRecords
for image, label, tri_label in tqdm(zip(test_images, test_one_hot_labels, test_tri_one_hot_labels), total=len(test_images)):
    feature = {
        'image': _bytes_feature(image),
        'label': _int64_feature(label),
        'tri_label': _int64_feature(tri_label)
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    test_writer.write(example.SerializeToString())

test_writer.close()


  9%|▉         | 178/1875 [00:00<00:01, 875.59it/s]

100%|██████████| 805/805 [00:00<00:00, 1450.39it/s]
